In [ ]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
os.chdir('D:/工作/data/titanic')
df = pd.read_csv('train.csv')
cat = df.select_dtypes(include='object')
enc = OneHotEncoder(categories='auto',handle_unknown='ignore')
cat = cat.fillna('None')
result = enc.fit_transform(cat)
label = LabelEncoder()
res = label.fit_transform(cat.iloc[:,2])

In [1]:
mapping={'x{}'.format(i):value for i, value in enumerate(cat.columns)}
pattern=re.compile(r'^x\d+')
d = map(lambda x: pattern.sub(mapping[pattern.match(x).group()],x),
        enc.get_feature_names())

In [ ]:
col = ['Survived', 'Pclass','Age','SibSp','Parch','Fare']
df.columns.difference(set(col))

In [7]:
def ks(preds,dtrain):
    label = dtrain.get_label()
    fpr, tpr, _ = roc_curve(label, preds)
    return 'ks', np.abs(fpr-tpr).max()

def lift(preds, dtrain):
    label = dtrain.get_label()
    groupor = pd.qcut(preds, q=10, duplicates='drop')
    label = pd.Series(label)
    result = label.groupby(groupor).mean()/label.mean()
    return 'lift',result.max()

In [29]:
Xt, Xv, yt, yv = train_test_split(X, y, test_size=0.2)

vdata = xgb.DMatrix(Xv, yv)
tdata = xgb.DMatrix(Xt, yt)
params['eval_metric']='auc'
res = {}
res1 ={}
result = xgb.train(params, tdata, evals=[(tdata, 'train'), (vdata, 'valid')],
          num_boost_round=200, feval=ks, evals_result=res, 
                   verbose_eval=False)
result = xgb.train(params, tdata, evals=[(tdata, 'train'), (vdata, 'valid')],
          num_boost_round=200, feval=lift, evals_result=res1, 
                   verbose_eval=False)

In [135]:
res['train'].update(res1['train'])
res['valid'].update(res1['valid'])
res['train'].keys()

dict_keys(['auc', 'ks', 'lift'])

In [92]:
params = xgb.XGBClassifier().get_xgb_params()
params['eval_metric']='auc'
res = {}
result = xgb.train(params, tdata, evals=[(tdata, 'train'), (vdata, 'valid')],
          num_boost_round=200, evals_result=res, 
                   verbose_eval=False)

In [66]:
ltdata= lgb.Dataset(Xt, yt)
lvdata = lgb.Dataset(Xv, yv)

In [74]:
def feval(preds, tdata):
    label = tdata.get_label()
    fpr, tpr, _ = roc_curve(label, preds)
    groupor = pd.qcut(preds, q=10, duplicates='drop')
    label = pd.Series(label)
    result = label.groupby(groupor).mean()/label.mean()
    return [('ks', np.abs(fpr-tpr).max(),True),('lift',result.max(), True)]

In [82]:
import lightgbm as lgb
es1 = lgb.LGBMClassifier()
params = es1.get_params()
params.pop('n_estimators')
params['metric'] = 'auc'

In [ ]:
ldic={}
result2 = lgb.train(params,ltdata,num_boost_round=200,
                    valid_sets=[ltdata,lvdata],valid_names=['train','valid'],
         feval=feval,verbose_eval=False,evals_result=ldic)

In [144]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar, Tab, Pie, Line
from pyecharts.globals import ThemeType
def res_process(res, res1):
    res['train'].update(res1['train'])
    res['valid'].update(res1['valid'])
    return res

def line(train, valid):
    index = int(np.argmax(valid)) 
    x = list(range(len(train)))
    c = (Line(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add_xaxis(list(range(200)))
        .add_yaxis("Train",ldic['train']['auc'],symbol='none',)
        .add_yaxis("Valid",ldic['valid']['auc'],symbol='none',
                   markline_opts=opts.MarkLineOpts(
                 data=[opts.MarkLineItem(name='auc',x=index)],
                   label_opts=opts.LabelOpts(is_show=False)))
        .set_global_opts(title_opts=opts.TitleOpts(),
                        xaxis_opts=opts.AxisOpts(type_="value")))
    return c
def tabshow(res):
    trains, valids=res['train'], res['valid']
    tab = Tab(page_title='模型选择')
    for key in trains.keys():
        tab.add(line(trains[key], valids[key]), key)
    return tab
tabshow(res).render_notebook()